# Can we beat Welch's t-test?

The current version of this script is super-stupid. But to do any interesting research, you have to plant
a few small acorns, and this is the smallest of acorns.

Suppose you have to evaluate NUMBER_OF_EXPERIMENTS experiments very small experiment. By a weird
coincidence, they all have the same number of participants. There are CONTROL_GROUP_SIZE patients
getting a placebo, and EXPERIMENT_GROUP_SIZE getting a drug.

Unfortunately approximately PROPORTION_OF_EXPERIMENTS_THAT_DO_NOTHING of those drugs actually have no effect either.

You want to create a classifier to determine whether or not the experimental drug worked.

Also, amazingly, the control group and the experimental group metrics are normally distributed. Obviously, the
experiments where the drug is useless have the same mean and standard deviation. In the experiments where
the drug is useful, they differ. The means of these datasets are uniformly distributed between 0-10. The standard deviations are uniformly distributed by 1-4.

Being a well-trained statistician with years of experience, you use Welch's t-test.

Your offsider is a clueless data scientist whose sole training consisted of reading the sklearn documentation.

You have a future self who will implement a beautifully trained transformer or fully-connected neural network. (I haven't done that in this code yet. One day soon.)

Who wins?

In [1]:
CONTROL_GROUP_SIZE=5
EXPERIMENT_GROUP_SIZE=6
PROPORTION_OF_EXPERIMENTS_THAT_DO_NOTHING=0.66
NUMBER_OF_EXPERIMENTS=10000

In [2]:
import scipy
import scipy.stats
import collections
import pandas
import sklearn.model_selection
import sklearn.svm
import sklearn.dummy
import sklearn.metrics
import sklearn.tree
import sklearn.ensemble

In [3]:
Experiment = collections.namedtuple("Experiment",
                                    ["ControlLoc", "ControlScale", "ControlValues",
                                   "ExperimentLoc", "ExperimentScale", "ExperimentValues", 
                                   "ShouldShowResult"])

In [4]:
def generate_experiment():
    null_experiment = scipy.stats.uniform.rvs() <= PROPORTION_OF_EXPERIMENTS_THAT_DO_NOTHING
    control_loc = scipy.stats.uniform.rvs(loc=0, scale=10)
    control_scale = scipy.stats.uniform.rvs(loc=1, scale=4)
    if null_experiment:
        experiment_loc = control_loc
        experiment_scale = control_scale
    else:
        experiment_loc = scipy.stats.uniform.rvs(loc=0, scale=10)
        experiment_scale = scipy.stats.uniform.rvs(loc=1, scale=4)
    control_values = scipy.stats.norm.rvs(loc=control_loc, scale=control_scale, size=CONTROL_GROUP_SIZE)
    experiment_values = scipy.stats.norm.rvs(loc=experiment_loc, scale=experiment_scale, size=EXPERIMENT_GROUP_SIZE)
    return Experiment(ControlLoc=control_loc, 
                      ControlScale=control_scale,
                     ControlValues=control_values,
                     ExperimentLoc=experiment_loc,
                     ExperimentScale=experiment_scale,
                     ExperimentValues=experiment_values,
                     ShouldShowResult=not(null_experiment))

def generate_experiments(number_of_experiments):
    return [generate_experiment() for n in range(number_of_experiments)]

In [5]:
def create_feature_dataframe(experiments):
    records = []
    for experiment in experiments:
        record = {}
        for i in range(CONTROL_GROUP_SIZE):
            record[f"c{i}"] = experiment.ControlValues[i]
        for i in range(EXPERIMENT_GROUP_SIZE):
            record[f"x{i}"] = experiment.ExperimentValues[i]
        records.append(record)
    return pandas.DataFrame.from_records(records)

def create_target_series(experiments):
    return pandas.Series([x.ShouldShowResult for x in experiments])

In [6]:
experiments = generate_experiments(NUMBER_OF_EXPERIMENTS)
create_feature_dataframe(experiments)

,c0,c1,c2,c3,c4,x0,x1,x2,x3,x4,x5
0,7.264473,-7.517165,11.771611,2.003438,-2.242615,6.937202,-0.366844,4.979517,5.467550,5.636427,-1.942541
1,7.157728,7.065377,7.153586,13.750535,5.320640,6.068166,7.299186,6.605873,9.193089,8.130642,7.989027
2,5.079459,6.833027,9.024674,6.761806,7.683872,3.239225,11.016344,6.166856,10.054226,14.344219,9.276773
3,4.655775,3.961034,-1.257782,-1.554726,3.558832,2.988543,-0.199298,-0.559302,0.512324,1.559637,1.981465
4,8.259506,9.068813,4.618367,7.748268,8.050739,8.516755,9.256255,3.781489,9.742290,8.329355,9.351403
...,...,...,...,...,...,...,...,...,...,...,...
9995,6.545834,8.056994,8.330863,7.300312,5.624735,7.742733,7.121092,7.251279,6.411926,6.741853,8.015904
9996,6.873099,2.160694,7.900879,5.117466,5.188634,9.712482,5.451041,7.427483,6.835240,6.573424,5.376153
9997,3.279704,3.586458,3.840117,3.032068,3.321618,5.283170,2.349917,5.647390,4.120839,1.540318,4.360842
9998,8.377527,11.193248,9.500909,3.416513,12.391999,1.263482,10.333170,7.608452,3.804797,10.732944,13.582453


In [7]:
create_target_series(experiments).mean()

0.3351

In [8]:
class WelchTTest:
    def __init__(self):
        pass
    def fit(self, X,y, weights=[]):
        pass
    def predict(self, Xs):
        answer = []
        for experiment in Xs:
            outcome = scipy.stats.ttest_ind(experiment.ControlValues, experiment.ExperimentValues, equal_var=False)
            if outcome.pvalue < 0.05:
                answer.append(True)
            else:
                answer.append(False)
        return answer

In [9]:
welch = WelchTTest()
welch_answers = welch.predict(experiments)
print(sklearn.metrics.confusion_matrix(create_target_series(experiments), welch_answers))

[[6334  315]
 [2105 1246]]


## The target to beat...

In [10]:
sklearn.metrics.accuracy_score(create_target_series(experiments), welch_answers)

0.758

## Various stupid ways to beat it

(That are all likely to fail...)

In [11]:
sklearn.model_selection.cross_validate(sklearn.dummy.DummyClassifier(strategy='most_frequent'), 
                                       create_feature_dataframe(experiments),
                                       create_target_series(experiments))['test_score'].mean()

0.6649

In [12]:
%%time
sklearn.model_selection.cross_validate(sklearn.ensemble.RandomForestClassifier(), 
                                       create_feature_dataframe(experiments),
                                       create_target_series(experiments))['test_score'].mean()

CPU times: user 9.93 s, sys: 41.2 ms, total: 9.97 s
Wall time: 10 s


0.7718999999999999

In [ ]:
%%time
sklearn.model_selection.cross_validate(sklearn.svm.SVC(kernel='rbf', C=1e9), 
                                       create_feature_dataframe(experiments),
                                       create_target_series(experiments))['test_score'].mean()

In [ ]:
#%%time
#sklearn.model_selection.cross_validate(sklearn.svm.SVC(kernel='poly', C=1e9, degree=2), 
##                                       create_feature_dataframe(experiments),
#                                       create_target_series(experiments))['test_score'].mean()

### Lost causes

In [ ]:
sklearn.model_selection.cross_validate(sklearn.svm.LinearSVC(dual=True, C=1e9), 
                                       create_feature_dataframe(experiments),
                                       create_target_series(experiments))['test_score'].mean()